# Extract Feature

## tutorial

In [2]:
import pandas as pd
import numpy as np

In [6]:
default_dir = '/home/users/s17011720/predicting_impression_using_mobile_game/'
tutorial = pd.read_csv(default_dir + "column_filtered_data/tutorial_filtered2.csv")

/home/users/s17011720/.conda/envs/jupyter/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (0,1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [126]:
day0_df = pd.DataFrame(tutorial.groupby('ad_id')['event_timestamp'].min())
day0_df.shape

(240911, 1)

In [133]:
day0_df.rename(columns={'event_timestamp':'day0'}, inplace=True)

In [128]:
day0_df.reset_index(inplace=True)

In [129]:
min_timestamp = pd.read_csv(default_dir + "column_filtered_data/min_timestamp.csv")

In [134]:
merge_df = pd.merge(se_start_df, day0_df, how='left', on=['ad_id'])

In [131]:
merge_df.shape

(558186, 4)

In [135]:
merge_df.head()

,ad_id,ga_session_id,event_timestamp,day0
0,00018501-6eee-45e9-b5b5-7b50f966abd8,1.618459e+09,1618458973665000,1618458973665000
1,00018501-6eee-45e9-b5b5-7b50f966abd8,1.619147e+09,1619147084357000,1618458973665000
2,0001edb6-e7e7-4f74-8993-2bfc9347a959,1.619207e+09,1619207127894000,1619207127894000
3,0001ef07-c27e-4d91-a8bb-d0aa5f0d45e7,1.618855e+09,1618854710317000,1618854710317000
4,0001ef07-c27e-4d91-a8bb-d0aa5f0d45e7,1.618932e+09,1618931879817000,1618854710317000


In [136]:
merge_df.to_csv(default_dir + "column_filtered_data/sess_start_and_day0.csv", index=False)

## asset

In [32]:
asset = pd.read_csv(default_dir + "column_filtered_data/asset_filtered.csv")
asset.head(2)

,event_previous_timestamp,event_timestamp,event_date,ad_id,user_pseudo_id,ga_session_id,ga_session_number,asset_id,asset_name,gubun,...,curr_var,action_id,action_name,item_id,item_name,stage,user_level,cuid,device_is_limited_ad_tracking,event_datetime
0,1.617937e+15,1617936778478001,20210408,37f17865-1a1b-4887-bf38-5ae581ec09cb,4370d11b9eae27ce9592bea98aedf391,1.617937e+09,1.0,1,coin,in,...,204,10001,play,0,0,4,4.0,24378877-1a21-40f2-9bfb-3d071076e551,False,2021-04-09 02:52:58.478001
1,1.618344e+15,1618344330246000,20210413,9a822b84-27d2-48c4-98d6-c87fa0a7b53a,686b15cf8de26e96faf785370a250d55,1.618344e+09,1.0,1,coin,out,...,306,20001,upgrade_atk,0,0,7,7.0,33afdb07-f374-4738-ae33-841e7c37e63e,False,2021-04-13 20:05:30.246000


In [33]:
asset.shape

(6231062, 22)

In [34]:
asset['event_datetime'] = pd.to_datetime(asset['event_timestamp'], unit='us').dt.date
asset.head(2)

,event_previous_timestamp,event_timestamp,event_date,ad_id,user_pseudo_id,ga_session_id,ga_session_number,asset_id,asset_name,gubun,...,curr_var,action_id,action_name,item_id,item_name,stage,user_level,cuid,device_is_limited_ad_tracking,event_datetime
0,1.617937e+15,1617936778478001,20210408,37f17865-1a1b-4887-bf38-5ae581ec09cb,4370d11b9eae27ce9592bea98aedf391,1.617937e+09,1.0,1,coin,in,...,204,10001,play,0,0,4,4.0,24378877-1a21-40f2-9bfb-3d071076e551,False,2021-04-09
1,1.618344e+15,1618344330246000,20210413,9a822b84-27d2-48c4-98d6-c87fa0a7b53a,686b15cf8de26e96faf785370a250d55,1.618344e+09,1.0,1,coin,out,...,306,20001,upgrade_atk,0,0,7,7.0,33afdb07-f374-4738-ae33-841e7c37e63e,False,2021-04-13


### target_user

In [6]:
def select_target(df):
    target = pd.read_csv(default_dir + "column_filtered_data/target.csv")
    target = target[target['not_target']==0]
    target_id = target['ad_id'].tolist()
    df = df[df['ad_id'].isin(target_id)]
    return df

def select_train(df, YN):
    target = pd.read_csv(default_dir + "column_filtered_data/target.csv")
    target = target[target['not_target']==0 & (df['train_YN']==1)]
    target_id = target['ad_id'].tolist()
    df = df[df['ad_id'].isin(target_id)]
    return df

In [148]:
target_asset = select_target(asset)
target_asset.shape

(6230295, 22)

In [149]:
len(target_asset.ad_id.unique()) #224,875

224875

### day_x 계산

In [7]:
default_dir = '/home/users/s17011720/predicting_impression_using_mobile_game/'

def calculate_day_x(df):
    day_df = pd.read_csv(default_dir + "column_filtered_data/sess_start_and_day0.csv")
    day_df['session_start_date'] = pd.to_datetime(day_df['event_timestamp'], unit='us')
    day_df['session_start_date'] = day_df.session_start_date.dt.date
    day_df['day0'] = pd.to_datetime(day_df['day0'], unit='us')
    day_df['day0'] = day_df.day0.dt.date
    day_df = day_df[['ad_id', 'ga_session_id', 'session_start_date', 'day0']]
    
    m_df = pd.merge(df, day_df, on=['ad_id', 'ga_session_id'])
    m_df['day_x'] = (m_df.session_start_date - m_df.day0).dt.days

    return m_df

In [163]:
asset_df = calculate_day_x(target_asset)

### 1. gubun : 코인 지급 in / out
- in_count, out_count

In [165]:
target_asset = asset_df.copy()
target_asset.shape

(6230095, 25)

In [166]:
grouped_gubun = target_asset.groupby(by=['ad_id','day_x','gubun'])
gubun = grouped_gubun.size().unstack()

In [168]:
gubun.head(2)

gubun                                        in  out
ad_id                                day_x          
00018501-6eee-45e9-b5b5-7b50f966abd8 0      1.0  NaN
                                     8      4.0  NaN

In [169]:
gubun.reset_index(inplace=True)
gubun.head(2)

gubun,ad_id,day_x,in,out
0,00018501-6eee-45e9-b5b5-7b50f966abd8,0,1.0,NaN
1,00018501-6eee-45e9-b5b5-7b50f966abd8,8,4.0,NaN


In [170]:
gubun.rename(columns = {'in':'in_coin_cnt',
                        'out':'out_coin_cnt'}, inplace = True)

In [171]:
gubun.shape

(416198, 4)

### 2. var : 실제 재화 변동량 (지급-양수, 회수-음수)
- in_sum(하루에 총 지급된 coin량 합산)
- out_sum(하루에 총 회수된 coin량 합산)

In [172]:
target_asset.loc[target_asset['var'] > 0, 'Payment'] = 'in'
target_asset.loc[target_asset['var'] < 0, 'Payment'] = 'out'

In [173]:
fluctuation = pd.DataFrame(target_asset.groupby(by=['ad_id','day_x','Payment'])['var'].sum().unstack())
fluctuation.reset_index(inplace=True)
fluctuation.head(2)

Payment,ad_id,day_x,in,out
0,00018501-6eee-45e9-b5b5-7b50f966abd8,0,51.0,NaN
1,00018501-6eee-45e9-b5b5-7b50f966abd8,8,254.0,NaN


In [174]:
fluctuation.rename(columns = {'in':'in_coin_sum',
                              'out':'out_coin_sum'}, inplace = True)

In [175]:
fluctuation['out_coin_sum'] = fluctuation['out_coin_sum'].abs()

### 3. curr_var : 현재 재화량
- 하루동안의 최소 coin (min)
- 하루동안의 최대 coin (max)

In [176]:
grouped_curr = target_asset.groupby(by=['ad_id', 'day_x'])
curr_var = grouped_curr['curr_var'].agg(['min', 'max'])

curr_var.reset_index(inplace=True)
curr_var.head(2)

,ad_id,day_x,min,max
0,00018501-6eee-45e9-b5b5-7b50f966abd8,0,51,51
1,00018501-6eee-45e9-b5b5-7b50f966abd8,8,102,305


In [177]:
curr_var.rename(columns = {'min':'curr_var_min',
                           'max':'curr_var_max'}, inplace = True)
curr_var.shape

(416198, 4)

### 4. action_id : coin지급지출과 관련된 액션의 종류
- action_id별 var의 sum  

In [178]:
# action_id별 var sum
grouped_curr = target_asset.groupby(by=['ad_id', 'day_x', 'action_name'])
var_sum_by_action = grouped_curr['var'].sum().unstack()

In [179]:
var_sum_by_action.reset_index(inplace=True)
var_sum_by_action.head(2)

action_name,ad_id,day_x,buy_skin,offline_coin,play,rv_collection_coin,rv_result_bonus_coin,rv_result_coin,upgrade_atk,upgrade_atk_speed,upgrade_coin,upgrade_hp
0,00018501-6eee-45e9-b5b5-7b50f966abd8,0,NaN,NaN,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00018501-6eee-45e9-b5b5-7b50f966abd8,8,NaN,NaN,254.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
var_sum_by_action.rename(columns = {'buy_skin':'buy_skin_var_sum',
                                    'offline_coin':'offline_coin_var_sum',
                                    'play':'play_var_sum',
                                    'rv_collection_coin':'rv_collection_coin_var_sum',
                                    'rv_result_bonus_coin':'rv_result_bonus_coin_var_sum',
                                    'rv_result_coin':'rv_result_coin_var_sum',
                                    'upgrade_atk':'upgrade_atk_var_sum',
                                    'upgrade_atk_speed':'upgrade_atk_speed_var_sum',
                                    'upgrade_coin':'upgrade_coin_var_sum',
                                    'upgrade_hp':'upgrade_hp_var_sum'}, 
                                    inplace = True)
# var_sum_by_action.head()

## 5. item_name별 count

In [181]:
asset['item_name'].unique()

array(['0', 'boy', 'starbucks', 'pringles', 'worker', 'firefighter',
       'suit', 'doctor', 'police'], dtype=object)

In [182]:
# item_name count
grouped_item_name = target_asset.groupby(by=['ad_id', 'day_x', 'item_name'])
count_by_action = grouped_item_name.size().unstack()

In [183]:
count_by_action.reset_index(inplace=True)
count_by_action.head(2)

item_name,ad_id,day_x,0,boy,doctor,firefighter,police,pringles,starbucks,suit,worker
0,00018501-6eee-45e9-b5b5-7b50f966abd8,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00018501-6eee-45e9-b5b5-7b50f966abd8,8,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Merge feature_df

In [184]:
asset_features = pd.merge(gubun, fluctuation, how='outer', on=['ad_id','day_x'])
print(asset_features.shape)
asset_features = pd.merge(asset_features, curr_var, how='outer', on=['ad_id','day_x'])
print(asset_features.shape)
asset_features = pd.merge(asset_features, var_sum_by_action, how='outer', on=['ad_id','day_x'])
print(asset_features.shape)
asset_features = pd.merge(asset_features, count_by_action, how='outer', on=['ad_id','day_x'])
print(asset_features.shape)
# (416198, 6)
# (416198, 8)
# (416198, 18)
# (416198, 27)

(416198, 6)
(416198, 8)
(416198, 18)
(416198, 27)


In [188]:
asset_features.columns

Index(['ad_id', 'day_x', 'in_coin_cnt', 'out_coin_cnt', 'in_coin_sum',
       'out_coin_sum', 'curr_var_min', 'curr_var_max', 'buy_skin_var_sum',
       'offline_coin_var_sum', 'play_var_sum', 'rv_collection_coin_var_sum',
       'rv_result_bonus_coin_var_sum', 'rv_result_coin_var_sum',
       'upgrade_atk_var_sum', 'upgrade_atk_speed_var_sum',
       'upgrade_coin_var_sum', 'upgrade_hp_var_sum', '0', 'boy', 'doctor',
       'firefighter', 'police', 'pringles', 'starbucks', 'suit', 'worker'],
      dtype='object')

In [190]:
asset_features.to_csv(default_dir + "extract_features_data/asset_features.csv", index=False)

## ads

In [1]:
default_dir = '/home/users/s17011720/predicting_impression_using_mobile_game/'

In [3]:
ads = pd.read_csv(default_dir + "column_filtered_data/ads_filtered.csv")
ads.head(2)

,event_previous_timestamp,event_timestamp,event_date,ad_id,user_pseudo_id,ga_session_id,ga_session_number,gubun,state,network,placement_id,placement_name,stage,Interval_time,user_level,cuid,device_is_limited_ad_tracking,event_datetime
0,1.618947e+15,1618947173023000,20210420,a39616e3-243c-4c04-b8d3-1a016111d84c,c52770eca550867afa956b2cb4e444b4,1.618947e+09,2.0,rv,view,on,2002,rv_result_coin,11,0,11,13685e42-2918-4cad-833f-daf8473c629a,False,2021-04-20 19:32:53.023000
1,1.618705e+15,1618705516341000,20210417,8abadb82-ab32-4031-9f66-441c2f6f8105,9735b62bab5187f9baee2769f393168e,1.618705e+09,1.0,rv,view,off,2002,rv_result_coin,10,0,10,0189da61-aac1-4e95-956a-70e9cdae87f5,False,2021-04-18 00:25:16.341000


In [4]:
ads.columns

Index(['event_previous_timestamp', 'event_timestamp', 'event_date', 'ad_id',
       'user_pseudo_id', 'ga_session_id', 'ga_session_number', 'gubun',
       'state', 'network', 'placement_id', 'placement_name', 'stage',
       'Interval_time', 'user_level', 'cuid', 'device_is_limited_ad_tracking',
       'event_datetime'],
      dtype='object')

In [5]:
ads = ads[['event_timestamp', 'ad_id','ga_session_id', 'gubun',
           'state', 'network', 'placement_name', 'Interval_time']]

In [8]:
target_ads = select_target(ads)

In [9]:
target_ads.shape

(17289413, 8)

In [10]:
len(target_ads.ad_id.unique()) #225,087

225087

In [11]:
ads_df = calculate_day_x(target_ads)

In [ ]:
ads[(ads['rv_imps']+ads['rv_imps_fail']) < ads['rv_color']][['rv_imps', 'rv_imps_fail', 'rv_color']]

In [34]:
ads_df[ads_df['placement_name'] == 'rv_color']

,event_timestamp,ad_id,ga_session_id,gubun,state,network,placement_name,Interval_time,session_start_date,day0,day_x
3,1618947803418000,a39616e3-243c-4c04-b8d3-1a016111d84c,1.618947e+09,rv,view,on,rv_color,0,2021-04-20,2021-04-20,0
6,1618947508248000,a39616e3-243c-4c04-b8d3-1a016111d84c,1.618947e+09,rv,view,on,rv_color,0,2021-04-20,2021-04-20,0
10,1618947129178000,a39616e3-243c-4c04-b8d3-1a016111d84c,1.618947e+09,rv,view,on,rv_color,0,2021-04-20,2021-04-20,0
13,1618947326971000,a39616e3-243c-4c04-b8d3-1a016111d84c,1.618947e+09,rv,view,on,rv_color,0,2021-04-20,2021-04-20,0
17,1618947744014000,a39616e3-243c-4c04-b8d3-1a016111d84c,1.618947e+09,rv,view,on,rv_color,0,2021-04-20,2021-04-20,0
...,...,...,...,...,...,...,...,...,...,...,...
17289036,1620398046189000,68ef7449-e3a6-44a7-a88b-dbae42bd36ef,1.620398e+09,rv,view,on,rv_color,0,2021-05-07,2021-04-30,7
17289038,1620828845355000,0e2e811f-3ec5-4b51-a9ae-4b7f4c8070d7,1.620829e+09,rv,view,on,rv_color,0,2021-05-12,2021-04-21,21
17289040,1620422492926000,df5eafb6-97f0-4ee0-9a53-eb81fa5a54c4,1.620422e+09,rv,view,on,rv_color,0,2021-05-07,2021-04-20,17
17289043,1618961873809000,65fdbbb0-bf5c-43b4-9391-06617c2f95cb,1.618962e+09,rv,view,off,rv_color,0,2021-04-20,2021-04-20,0


### 1. gubun

In [13]:
ads_df.columns

Index(['event_timestamp', 'ad_id', 'ga_session_id', 'gubun', 'state',
       'network', 'placement_name', 'Interval_time', 'session_start_date',
       'day0', 'day_x'],
      dtype='object')

In [12]:
ads_df.head()

,event_timestamp,ad_id,ga_session_id,gubun,state,network,placement_name,Interval_time,session_start_date,day0,day_x
0,1618947173023000,a39616e3-243c-4c04-b8d3-1a016111d84c,1.618947e+09,rv,view,on,rv_result_coin,0,2021-04-20,2021-04-20,0
1,1618947395410000,a39616e3-243c-4c04-b8d3-1a016111d84c,1.618947e+09,rv,view,on,rv_result_bonus_coin,0,2021-04-20,2021-04-20,0
2,1618947555551000,a39616e3-243c-4c04-b8d3-1a016111d84c,1.618947e+09,ba,imps,on,ba_ingame,0,2021-04-20,2021-04-20,0
3,1618947803418000,a39616e3-243c-4c04-b8d3-1a016111d84c,1.618947e+09,rv,view,on,rv_color,0,2021-04-20,2021-04-20,0
4,1618947655016000,a39616e3-243c-4c04-b8d3-1a016111d84c,1.618947e+09,ba,imps,on,ba_ingame,0,2021-04-20,2021-04-20,0


In [13]:
imps = ads_df.groupby(['ad_id', 'day_x', 'state', 'gubun']).size()
imps = imps.unstack().unstack()
imps.head(2)

gubun                                        ba                  is            \
state                                      imps imps_fail view imps imps_fail   
ad_id                                day_x                                      
00018501-6eee-45e9-b5b5-7b50f966abd8 0      1.0       NaN  NaN  NaN       NaN   
                                     8      NaN       NaN  NaN  NaN       3.0   

gubun                                             rv                 
state                                      view imps imps_fail view  
ad_id                                day_x                           
00018501-6eee-45e9-b5b5-7b50f966abd8 0      NaN  NaN       NaN  1.0  
                                     8      NaN  NaN       3.0  8.0

In [15]:
imps.columns = ['_'.join(col).strip() for col in imps.columns.values]

In [18]:
imps.reset_index(inplace=True)
imps.head(2)

,ad_id,day_x,ba_imps,ba_imps_fail,ba_view,is_imps,is_imps_fail,is_view,rv_imps,rv_imps_fail,rv_view
0,00018501-6eee-45e9-b5b5-7b50f966abd8,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,00018501-6eee-45e9-b5b5-7b50f966abd8,8,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.0,8.0


In [20]:
imps_gubun = ads_df.groupby(['ad_id', 'day_x', 'network', 'state', 'gubun']).size()
# imps_gubun.head(2)

In [21]:
imps_gubun=imps_gubun.unstack().unstack().unstack()
imps_gubun.head(2)

gubun                                        ba                              \
state                                      imps      imps_fail     view       
network                                     off   on       off  on  off  on   
ad_id                                day_x                                    
00018501-6eee-45e9-b5b5-7b50f966abd8 0      NaN  1.0       NaN NaN  NaN NaN   
                                     8      NaN  NaN       NaN NaN  NaN NaN   

gubun                                        is                              \
state                                      imps     imps_fail      view       
network                                     off  on       off   on  off  on   
ad_id                                day_x                                    
00018501-6eee-45e9-b5b5-7b50f966abd8 0      NaN NaN       NaN  NaN  NaN NaN   
                                     8      NaN NaN       NaN  3.0  NaN NaN   

gubun                                        rv                               
state                                      imps     imps_fail      view       
network                                     off  on       off   on  off   on  
ad_id                                day_x                                    
00018501-6eee-45e9-b5b5-7b50f966abd8 0      NaN NaN       NaN  NaN  NaN  1.0  
                                     8      NaN NaN       NaN  3.0  1.0  7.0

In [22]:
imps_gubun.columns = ['_'.join(col).strip() for col in imps_gubun.columns.values]

In [23]:
imps_gubun.reset_index(inplace=True)
imps_gubun.head(2)

,ad_id,day_x,ba_imps_off,ba_imps_on,ba_imps_fail_off,ba_imps_fail_on,ba_view_off,ba_view_on,is_imps_off,is_imps_on,is_imps_fail_off,is_imps_fail_on,is_view_off,is_view_on,rv_imps_off,rv_imps_on,rv_imps_fail_off,rv_imps_fail_on,rv_view_off,rv_view_on
0,00018501-6eee-45e9-b5b5-7b50f966abd8,0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,00018501-6eee-45e9-b5b5-7b50f966abd8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.0,1.0,7.0


### 2. placement_id : 광고 노출 위치 아이디
- id별 하루동안의 광고 성공수(imps) 카운트 합산

In [24]:
ads_df.columns

Index(['event_timestamp', 'ad_id', 'ga_session_id', 'gubun', 'state',
       'network', 'placement_name', 'Interval_time', 'session_start_date',
       'day0', 'day_x'],
      dtype='object')

In [25]:
imps_placement = ads_df.groupby(['ad_id', 'day_x', 'placement_name']).size().unstack()
imps_placement.head(2)

placement_name                              ba_home  ba_ingame  ba_result  \
ad_id                                day_x                                  
00018501-6eee-45e9-b5b5-7b50f966abd8 0          1.0        NaN        NaN   
                                     8          NaN        NaN        NaN   

placement_name                              is_death  is_game_end  \
ad_id                                day_x                          
00018501-6eee-45e9-b5b5-7b50f966abd8 0           NaN          NaN   
                                     8           NaN          3.0   

placement_name                              rv_collection_coin  rv_color  \
ad_id                                day_x                                 
00018501-6eee-45e9-b5b5-7b50f966abd8 0                     NaN       NaN   
                                     8                     NaN       4.0   

placement_name                              rv_result_bonus_coin  \
ad_id                                day_x                         
00018501-6eee-45e9-b5b5-7b50f966abd8 0                       NaN   
                                     8                       NaN   

placement_name                              rv_result_coin  rv_up_1  rv_up_2  \
ad_id                                day_x                                     
00018501-6eee-45e9-b5b5-7b50f966abd8 0                 1.0      NaN      NaN   
                                     8                 7.0      NaN      NaN   

placement_name                              rv_up_3  rv_up_4  
ad_id                                day_x                    
00018501-6eee-45e9-b5b5-7b50f966abd8 0          NaN      NaN  
                                     8          NaN      NaN

In [26]:
imps_placement.reset_index(inplace=True)
imps_placement.head(2)

placement_name,ad_id,day_x,ba_home,ba_ingame,ba_result,is_death,is_game_end,rv_collection_coin,rv_color,rv_result_bonus_coin,rv_result_coin,rv_up_1,rv_up_2,rv_up_3,rv_up_4
0,00018501-6eee-45e9-b5b5-7b50f966abd8,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1,00018501-6eee-45e9-b5b5-7b50f966abd8,8,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN,7.0,NaN,NaN,NaN,NaN


### 3. Interval time

In [27]:
Interval_time = pd.DataFrame(ads_df.groupby(['ad_id', 'day_x'])['Interval_time'].sum())
Interval_time.head(2)

Interval_time
ad_id                                day_x               
00018501-6eee-45e9-b5b5-7b50f966abd8 0                  0
                                     8                  0

In [28]:
Interval_time.reset_index(inplace=True)
Interval_time.head(2)

,ad_id,day_x,Interval_time
0,00018501-6eee-45e9-b5b5-7b50f966abd8,0,0
1,00018501-6eee-45e9-b5b5-7b50f966abd8,8,0


In [29]:
Interval_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419923 entries, 0 to 419922
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ad_id          419923 non-null  object
 1   day_x          419923 non-null  int64 
 2   Interval_time  419923 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.6+ MB


## Merge Feature_df

In [30]:
ads_features = pd.merge(imps_gubun, imps_placement, how='outer', on=['ad_id','day_x'])
print(ads_features.shape)
ads_features = pd.merge(ads_features, imps, how='outer', on=['ad_id','day_x'])
print(ads_features.shape)
ads_features = pd.merge(ads_features, Interval_time, how='outer', on=['ad_id','day_x'])
print(ads_features.shape)

# (419923, 33)

(419923, 33)
(419923, 42)
(419923, 43)


In [31]:
ads_features.head()

,ad_id,day_x,ba_imps_off,ba_imps_on,ba_imps_fail_off,ba_imps_fail_on,ba_view_off,ba_view_on,is_imps_off,is_imps_on,...,ba_imps,ba_imps_fail,ba_view,is_imps,is_imps_fail,is_view,rv_imps,rv_imps_fail,rv_view,Interval_time
0,00018501-6eee-45e9-b5b5-7b50f966abd8,0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
1,00018501-6eee-45e9-b5b5-7b50f966abd8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.0,8.0,0
2,0001edb6-e7e7-4f74-8993-2bfc9347a959,0,NaN,12.0,NaN,NaN,NaN,NaN,NaN,7.0,...,12.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,26.0,298
3,0001ef07-c27e-4d91-a8bb-d0aa5f0d45e7,0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,2.0,...,4.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,6.0,98
4,0001ef07-c27e-4d91-a8bb-d0aa5f0d45e7,1,NaN,8.0,NaN,NaN,NaN,NaN,NaN,3.0,...,8.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,10.0,151


In [32]:
ads_features.to_csv(default_dir + "extract_features_data/ads_features.csv", index=False)

## activity

In [231]:
activity = pd.read_csv(default_dir + "column_filtered_data/activity_filtered.csv")
activity.head(2)

,event_previous_timestamp,event_timestamp,event_date,ad_id,user_pseudo_id,ga_session_id,ga_session_number,activity_id,activity_name,action_id,action_name,sub_id,sub_name,user_level,cuid,device_is_limited_ad_tracking,event_datetime
0,1.619310e+15,1619310355712000,20210424,c9000a27-6fd5-4eb1-b728-286561c707ca,8da97c0b43305c41d78cc0f5585d6b9a,1.619309e+09,11.0,3,content_button,3003,click_skin,1008,firefighter,111,2c8601cf-c8a7-4046-9edf-f5a81463ec60,False,2021-04-25 00:25:55.712000
1,1.619311e+15,1619310716820000,20210424,c9000a27-6fd5-4eb1-b728-286561c707ca,8da97c0b43305c41d78cc0f5585d6b9a,1.619309e+09,11.0,3,content_button,3002,close_collection,0,0,116,2c8601cf-c8a7-4046-9edf-f5a81463ec60,False,2021-04-25 00:31:56.820000


In [232]:
activity.columns

Index(['event_previous_timestamp', 'event_timestamp', 'event_date', 'ad_id',
       'user_pseudo_id', 'ga_session_id', 'ga_session_number', 'activity_id',
       'activity_name', 'action_id', 'action_name', 'sub_id', 'sub_name',
       'user_level', 'cuid', 'device_is_limited_ad_tracking',
       'event_datetime'],
      dtype='object')

In [235]:
activity = activity[['event_timestamp', 'ad_id', 'ga_session_id', 
                    'activity_name', 'action_name', 'sub_name']]

In [238]:
target_act = select_target(activity)

In [239]:
target_act.shape

(1081160, 6)

In [240]:
len(target_act.ad_id.unique()) #144,083

144083

In [241]:
act_df = calculate_day_x(target_act)

### 1. 해당 날짜에 activity를 한 횟수 (count)

In [246]:
act_count = pd.DataFrame(act_df.groupby(['ad_id', 'day_x']).size())

In [247]:
act_count.head(2)

,,0
ad_id,day_x,
0001edb6-e7e7-4f74-8993-2bfc9347a959,0,1
0002cac0-644a-4169-8253-6e02cf634b7b,0,1


In [248]:
act_count.rename(columns={0:'act_cnt'}, inplace=True)

,,act_cnt
ad_id,day_x,
0001edb6-e7e7-4f74-8993-2bfc9347a959,0,1
0002cac0-644a-4169-8253-6e02cf634b7b,0,1


In [249]:
act_count.reset_index(inplace=True)
act_count.head(2)

,ad_id,day_x,act_cnt
0,0001edb6-e7e7-4f74-8993-2bfc9347a959,0,1
1,0002cac0-644a-4169-8253-6e02cf634b7b,0,1


### 2. activity_id별 누른 횟수
activity_id : activity_name

- 1 : rating_popup / ~~2 : iap_click~~ / 3 : content_click

In [258]:
activity_cnt = act_df.groupby(['ad_id', 'day_x','activity_name']).size().unstack()
activity_cnt.head(2)

,activity_name,content_button,rating_popup
ad_id,day_x,,
0001edb6-e7e7-4f74-8993-2bfc9347a959,0,NaN,1.0
0002cac0-644a-4169-8253-6e02cf634b7b,0,NaN,1.0


In [259]:
activity_cnt.reset_index(inplace=True)

### 3. action_id 별 누른 횟수
action_id : action_name

- activity_id = 1\
1001 : yes / 1002 : no / 1003 : exit

- activity_id = 2\
데이터 없음

- activity_id = 3\
3001 : open_collection / 3002 : close collection / 3003 : click_skin (3003 0 -> sub_id : sub_name = skin_id : skin_name

In [260]:
action_cnt = act_df.groupby(['ad_id', 'day_x','action_name']).size().unstack()
action_cnt.head(2)

,action_name,Yes,click_skin,close_collection,open_collection
ad_id,day_x,,,,
0001edb6-e7e7-4f74-8993-2bfc9347a959,0,1.0,NaN,NaN,NaN
0002cac0-644a-4169-8253-6e02cf634b7b,0,1.0,NaN,NaN,NaN


In [265]:
action_cnt.reset_index(inplace=True)

In [267]:
action_cnt.head()

action_name,ad_id,day_x,Yes,click_skin,close_collection,open_collection
0,0001edb6-e7e7-4f74-8993-2bfc9347a959,0,1.0,NaN,NaN,NaN
1,0002cac0-644a-4169-8253-6e02cf634b7b,0,1.0,NaN,NaN,NaN
2,00031e14-e86d-49ac-bda0-a1427e9dcd3b,0,1.0,NaN,NaN,NaN
3,00038635-61d2-4656-b04b-332c5a6755cd,0,1.0,5.0,1.0,1.0
4,00038635-61d2-4656-b04b-332c5a6755cd,1,1.0,5.0,2.0,2.0


In [269]:
action_cnt.drop('Yes', axis=1, inplace=True)

## Merge feature_df

In [270]:
act_features = pd.merge(act_count, activity_cnt, how='outer', on=['ad_id','day_x'])
print(act_features.shape)
act_features = pd.merge(act_features, action_cnt, how='outer', on=['ad_id','day_x'])
print(act_features.shape)
# (224136, 5)
# (224136, 8)

(224136, 5)
(224136, 8)


In [271]:
act_features.to_csv(default_dir + "extract_features_data/act_features.csv", index=False)